In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pathlib import Path
from scipy.optimize import curve_fit
import math

In [6]:
def df_process(filepath):

    df = pd.read_csv(filepath, header=[0,1], index_col=0)

    # Prepare data
    df.index.name = 'Sample'
    df.columns.names = ['Replicate', 'Wavelength']
    df_long = df.stack(level='Replicate').reset_index()
    df_long['replicate'] = df_long['Replicate'].str.extract(r'rep(\d+)').astype(int)
    df_long = df_long.drop(columns=['Replicate'])

    # Sort by replicate (1, then 2, then 3) and Sample within each group
    df_long = df_long.sort_values(['replicate', 'Sample']).reset_index(drop=True)

    return df_long

In [10]:
base_dirs = ['20250725_1131']
# 

pattern = re.compile(
    r'^CAPB_'                 # 固定前缀
    r'(?P<time>\d+)min_'      # 数字 + "min_"
    r'(?P<assay>[abc])'       # 只允许 a, b, c
    r'_raw_multiindex\.csv$'  # 严格匹配 ".csv" 并用 $ 尾
)


# 1) Read & combine output_data
out_dfs = []
for base in base_dirs:
    for path in (Path(base) / 'raw_data').glob('*.csv'):
        m = pattern.match(path.name)
        if not m:
            continue

        t    = int(m.group('time'))
        assay  = m.group('assay')


        df = pd.df_process(path)
        print(df.describe())
#        df['ratio']      = df['334_373'] / df['334_384']
        df['assay']      = assay
        df['time']        = t

        out_dfs.append(df)

# combined_raw = pd.concat(out_dfs, ignore_index=True)

# # give each measurement a “row-within-group” index
# combined_raw['well_idx'] = (
#     combined_raw
#       .groupby(['surfactant','time','assay','replicate'])
#       .cumcount()
# )



In [3]:
max_conc = 50
estimate_cmc = 0.627
number_of_points = 12

max_conc = max_conc * 0.8  # Allowing for a bit of extra space
low = estimate_cmc / (50**0.5)
high = estimate_cmc * (50**0.5)

if high > max_conc:
    print(f"Warning: High concentration {high} exceeds maximum stock concentration {max_conc}. Adjusting high to max stock concentration.")
    high = max_conc


exponent_low = np.round(np.log10(low), 3)
exponent_high = np.round(np.log10(high), 3)
concentrations = np.round(np.logspace(exponent_low, exponent_high, number_of_points), 3)

concentrations

array([0.089, 0.127, 0.181, 0.258, 0.368, 0.525, 0.749, 1.07 , 1.526,
       2.178, 3.108, 4.436])